In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import glob

from dipy.core.sphere import faces_from_sphere_vertices, unique_edges
from dipy.data import get_sphere, HemiSphere, Sphere
from dipy.io.streamline import load_trk
from dipy.io.streamline import save_trk
from dipy.tracking import utils
from dipy.tracking.local import (CmcTissueClassifier, LocalTracking)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask
from dipy.direction import ProbabilisticDirectionGetter, DeterministicMaximumDirectionGetter
from os import path

In [2]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing')
wd = '/space/hemera/1/users/cmaffei/hcp_processing'

In [3]:
sphere642 = get_sphere('symmetric642')
sphere642.vertices = sphere642.vertices.astype('float')
sphere642.edges = sphere642.edges.astype('uint16')
sphere642.faces = sphere642.faces.astype('uint16')
sphere642.phi = sphere642.phi.astype('float')

## GQI Tractography

In [ ]:
#tractography parameters
step_size = 0.25
max_angle= 30
pmf_thr=0.5
seeds_count=10
affine=np.eye(4)

for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    if path.exists('gqi_prob_npv10_30_025.trk'):
        print('Tractography already done')
    else:
        print('Loading ODFs for'+str(sub)) #loading odfs
        img = nib.load(os.path.join(sub, 'all_shells/odfs_norm.nii.gz'))
        odfs_gqi = img.get_data()
        #Getting directions for tracking
        pmf = odfs_gqi.clip(min=0)
        prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle, 
                                                    sphere642, pmf_thr)
        #Define the seeds from a white matter binary mask
        seed_mask = nib.load(os.path.join(sub, 'wm_mask_bin.nii.gz')).get_data()    
        seeds = random_seeds_from_mask(seed_mask, seeds_count)
        #Import PVE maps
        img_pve_gm =  nib.load(os.path.join(sub, 't12fa_pve_1_res.nii.gz'))
        img_pve_csf =  nib.load(os.path.join(sub, 't12fa_pve_0_res.nii.gz'))
        img_pve_wm = nib.load(os.path.join(sub, 't12fa_pve_2_res.nii.gz'))
        voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
        #cmc_classifier
        cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)
        print('Start Tracking for'+str(sub)) #tracking
        all_streamlines_cmc_classifier = LocalTracking(prob_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine, step_size=step_size,
                                               return_all=True)
        streamlines = Streamlines(all_streamlines_cmc_classifier)
        print ('Saving Streamlines for'+str(sub))#save_trk
        save_trk(os.path.join(sub, 'gqi_prob_npv10_30_025.trk'), streamlines, img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

## GQI Deterministic

In [5]:
#tractography parameters
step_size = 0.25
max_angle= 30
seeds_count=10
affine=np.eye(4)

for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    if path.exists(os.path.join(sub, 'gqi_det_npv10_30_025.trk')):
        print('Tractography already done')
    else:
        print('Loading ODFs for'+str(sub)) #loading odfs
        img = nib.load(os.path.join(sub, 'all_shells/odfs_norm.nii.gz'))
        odfs_gqi = img.get_data()
        pmf = odfs_gqi.clip(min=0)
        det_dg = DeterministicMaximumDirectionGetter.from_pmf(pmf,max_angle,
                                                             sphere642)
        #Define the seeds from a white matter binary mask
        seed_mask = nib.load(os.path.join(sub, 'wm_mask_bin.nii.gz')).get_data()    
        seeds = random_seeds_from_mask(seed_mask, seeds_count)
        #Import PVE maps
        img_pve_gm =  nib.load(os.path.join(sub, 't12fa_pve_1_res.nii.gz'))
        img_pve_csf =  nib.load(os.path.join(sub, 't12fa_pve_0_res.nii.gz'))
        img_pve_wm = nib.load(os.path.join(sub, 't12fa_pve_2_res.nii.gz'))
        voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
        #cmc_classifier
        cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)
        print('Start Tracking for'+str(sub)) #tracking
        all_streamlines_cmc_classifier = LocalTracking(det_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine, step_size,
                                               return_all=True)
        streamlines = Streamlines(all_streamlines_cmc_classifier)
        print ('Saving Streamlines for'+str(sub))#save_trk
        save_trk(os.path.join(sub, 'gqi_det_npv10_30_025.trk'), streamlines, img.affine,
             shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Tractography already done
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1030


/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1030
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1030
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1013
Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1013
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1013
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1034
Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1034
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1034
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1025
Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1025
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1025
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1009
Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1009
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_

## CSD Tractography

In [3]:
#tractography parameters
step_size = 0.5
max_angle= 30
pmf_thr=0.1
seeds_count=2
affine=np.eye(4)
sphere = get_sphere('symmetric724')

for sub in sorted(glob.glob(os.path.join(wd, 'mgh_1010'))):
        print('Loading ODFs for'+str(sub)) #loading odfs
        img = nib.load(os.path.join(sub, '10k/odfs_01_01_body_cc.nii.gz'))
        odfs_csd = img.get_data()
        #Getting directions for tracking
        prob_dg = ProbabilisticDirectionGetter.from_shcoeff(odfs_csd, max_angle, 
                                                    sphere, pmf_thr)
        #Define the seeds from a white matter binary mask
        seed_mask = nib.load(os.path.join(sub, 't12fa_n_pve_2_res.nii.gz')).get_data()
        seed_mask = seed_mask > 0.5
        seeds = random_seeds_from_mask(seed_mask, seeds_count)
        #Import PVE maps
        img_pve_gm =  nib.load(os.path.join(sub, 't12fa_n_pve_1_res.nii.gz'))
        img_pve_csf =  nib.load(os.path.join(sub, 't12fa_n_pve_0_res.nii.gz'))
        img_pve_wm = nib.load(os.path.join(sub, 't12fa_n_pve_2_res.nii.gz'))
        voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
        #cmc_classifier
        cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)
        print('Start Tracking for'+str(sub)) #tracking
        all_streamlines_cmc_classifier = LocalTracking(prob_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine, step_size=step_size,
                                               return_all=True)
        streamlines = Streamlines(all_streamlines_cmc_classifier)
        print ('Saving Streamlines for'+str(sub))#save_trk
        save_trk(os.path.join(sub, '10k/odfs_01_01_body_cc_30_05_n_npv2.trk'), streamlines, img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])
    

Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1010


/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


Start Tracking for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1010
Saving Streamlines for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1010


In [ ]:
streamline_generator = LocalTracking(
        csd_peaks, tissue_classifier, seeds, affine=np.eye(4),
        step_size=0.5)